<a href="https://colab.research.google.com/github/hanseul1215/ESAA_study/blob/master/%ED%91%B8%EB%A6%AC%EC%97%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/bio/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/bio/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/bio/sample_submission.csv')

# 결측치 보간

In [ ]:
import numpy as np

train_dst = train.filter(regex = '_dst$', axis = 1).replace(0, np.NaN)
test_dst = test.filter(regex = '_dst$', axis = 1).replace(0, np.NaN)

In [ ]:
train_dst = train_dst.interpolate(method = 'linear', axis = 1)
test_dst = test_dst.interpolate(method = 'linear', axis = 1)

In [ ]:
train_dst.fillna(0, inplace = True) 
test_dst.fillna(0, inplace = True)

In [ ]:
train.update(train_dst)
test.update(test_dst)

# 특성공학

In [ ]:
feature = list(test)
target = ['hhb', 'hbo2', 'ca', 'na']

X_train = train[feature]
X_test = test[feature]

y_train = train[target]


df = pd.concat([X_train, X_test] , axis=0)

### 1) 투과도 : dst/src

In [ ]:
for i in range(650,1000,10) : 
    df['투과도_{}'.format(i)] =  (df['{}_dst'.format(i)]) / (df['{}_src'.format(i)])


### 2) 흡광계수 : log(src/dst)/rho

In [ ]:
for i in range(650,1000,10) : 
    df['흡광계수_{}'.format(i)] = np.log10(df['{}_src'.format(i)] / df['{}_dst'.format(i)]) / df.rho

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
for i in range(650,1000,10):
    df['투과도_{}'.format(i)]= df['투과도_{}'.format(i)].replace(np.inf,0).replace(-np.inf,0)
    df['흡광계수_{}'.format(i)]=df['흡광계수_{}'.format(i)].replace(np.inf,0).replace(-np.inf,0)

### 3) 푸리에 변환

In [ ]:
from tqdm import tqdm
df_imag = df.copy()

dst_list = list(df.loc[:,'650_dst':'990_dst'].columns)
df_real=df[dst_list] # 실수부
df_imag=df[dst_list] # 허수부

for i in tqdm(df_real.index): # tqdm() : 진행률 프로세스 바 
    df_real.loc[i]=df_real.loc[i] - df_real.loc[i].mean()
    df_imag.loc[i]=df_imag.loc[i] - df_imag.loc[i].mean()
    
    df_real.loc[i] = np.fft.fft(df_real.loc[i], norm='ortho').real # ortho : 퓨리에 값 정규화
    df_imag.loc[i] = np.fft.fft(df_imag.loc[i], norm='ortho').imag

real_part=[]
imag_part=[]

for col in dst_list:
    real_part.append(col + '_fft_real')
    imag_part.append(col + '_fft_imag')
    
df_real.columns=real_part
df_imag.columns=imag_part
df_p = pd.concat((df_real, df_imag), axis=1)


df=pd.concat((df, df_p), axis=1)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

# 모델링

In [ ]:
y_list = train[['hhb', 'hbo2', 'ca', 'na']]
X_train=df[:len(y_list)]
X_test=df[len(y_list):]
y_train=y_list

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imputer = IterativeImputer(random_state = 100)
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

## optuna -> lgbm

### LightGBM

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score, StratifiedKFold


def model_scoring_cv(model, x, y, cv =5 ):
    score = - cross_val_score(model, x, y, cv = cv, scoring = 'neg_mean_absolute_error').mean()
    return score

In [ ]:
lgbm1=LGBMRegressor(n_estimators= 900, learning_rate= 0.044603166889774906, num_leaves= 180, max_depth= 15, 
                    min_child_samples= 35, lambda_l1= 0, lambda_l2= 45, feature_fraction= 0.2, random_state= 2021)

lgbm1.fit(X_train, y_train.iloc[:,0])

lgbm2=LGBMRegressor(n_estimators= 900, learning_rate= 0.05099515681755944, num_leaves= 450, max_depth= 17, 
                    min_child_samples= 40, lambda_l1= 0, lambda_l2= 75, feature_fraction = 0.30000000000000004, random_state= 2021)

lgbm2.fit(X_train, y_train.iloc[:,1])

lgbm3=LGBMRegressor(n_estimators= 900, learning_rate= 0.04118535923202951, num_leaves= 500, max_depth= 16, 
                    min_child_samples= 25, lambda_l1= 0, lambda_l2= 30, feature_fraction= 0.7,random_state=2021)

lgbm3.fit(X_train, y_train.iloc[:,2])

lgbm4=LGBMRegressor(n_estimators= 500, learning_rate= 0.06267782322281297, num_leaves= 500, max_depth= 20, 
                    min_child_samples= 50, lambda_l1= 0, lambda_l2= 75, feature_fraction= 0.2,random_state=2021)

lgbm4.fit(X_train, y_train.iloc[:,3])

LGBMRegressor(feature_fraction=0.2, lambda_l1=0, lambda_l2=45,
              learning_rate=0.044603166889774906, max_depth=15,
              min_child_samples=35, n_estimators=900, num_leaves=180,
              random_state=2021)

In [ ]:
pred1 = pd.DataFrame(lgbm1.predict(X_test))
pred2 = pd.DataFrame(lgbm2.predict(X_test))
pred3 = pd.DataFrame(lgbm3.predict(X_test))
pred4 = pd.DataFrame(lgbm4.predict(X_test))

In [ ]:
pred = pd.concat([pred1, pred2, pred3, pred4], axis=1)
pred.columns= ['hhb', 'hbo2', 'ca', 'na']
pred.head()

,hhb,hbo2,ca,na
0,8.608140,4.956481,10.357565,3.118878
1,6.851746,2.948686,8.889655,3.256343
2,9.663804,4.937365,13.188400,3.600806
3,8.289608,3.781732,8.460574,4.598409
4,5.948220,3.437717,9.097053,2.735458


In [ ]:
submission = pd.concat([submission["id"], pred], axis=1)

In [ ]:
submission

,id,hhb,hbo2,ca,na
0,10000,8.608140,4.956481,10.357565,3.118878
1,10001,6.851746,2.948686,8.889655,3.256343
2,10002,9.663804,4.937365,13.188400,3.600806
3,10003,8.289608,3.781732,8.460574,4.598409
4,10004,5.948220,3.437717,9.097053,2.735458
...,...,...,...,...,...
9995,19995,4.026966,4.354015,11.812105,4.723345
9996,19996,10.673458,2.629203,6.621760,3.491386
9997,19997,7.199531,3.850147,7.944539,2.918365
9998,19998,12.089985,4.708429,9.867784,3.896260


In [ ]:
submission.to_csv('푸리에.csv', index=False) 